In [ ]:
import json
import os
import random
import fnmatch
import openpyxl

from urllib.request import urlopen
import urllib

import pandas as pd
import numpy as np
import re

In [ ]:
propilot_path = os.path.join("data")

df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [ ]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

In [ ]:
import re
def clean_string(s):
    s = re.sub('\xa0', ' ', s)
    s = re.sub('\.', ' ', s)
    s = re.sub(' +', ' ', s)
    return s.strip()


tree_nodes = df_dict["dim_tree_nodes"].copy()
tree_nodes['code_mesure'] = tree_nodes['tree_node_code'].apply(lambda x: x[:3])
tree_nodes['mesure'] = tree_nodes['tree_node_name'].apply(lambda x: clean_string(' / '.join(x.split('/')[1:])) if '/' in x else x)

In [ ]:
df_mesures = tree_nodes.groupby(['code_mesure', 'mesure'])['maturity_id'].apply(tuple).reset_index()[['code_mesure', 'mesure', 'maturity_id']]

In [ ]:
mi_ref = pd.DataFrame()
mi_ref['indicateur'] = df_dict["fact_financials"][~df_dict["fact_financials"]['effect_id'].isnull()]['effect_id'].unique()
mi_ref['code_mesure'] = mi_ref['indicateur'].apply(lambda x: x[-3:])
mi_ref = mi_ref.merge(df_mesures[['mesure', 'code_mesure']], left_on='code_mesure', right_on='code_mesure', how='left')


In [ ]:
mi_ref.shape

In [ ]:
mi_ref

In [ ]:
!mkdir refs

In [ ]:
mi_ref.to_csv('refs/ref_indicateurs.csv', sep=';', index=False)